# Doc2Vec

In [ ]:
pip uninstall numpy gensim -y

In [ ]:
pip install --upgrade "numpy>=2.0.0,<2.1.0"

In [ ]:
pip install --upgrade gensim

In [1]:
import numpy as np
import gensim
#import thinc

print(f"NumPy version: {np.__version__}") # 2.0.x 버전이 출력되어야 합니다
print(f"Gensim version: {gensim.__version__}")
#print(f"Thinc version: {thinc.__version__}")
print("All imports successful!")

NumPy version: 1.26.4
Gensim version: 4.3.3
All imports successful!


In [2]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm
import glob
import os

In [4]:
# 파일 열기
with open('./postagging/tagged_contents_df_0510.pkl', 'rb') as f:
    df = pickle.load(f)

# 데이터프레임 합치기
df = pd.DataFrame(df)

# 확인
print(f"✅ df.shape: {df.shape}")

✅ df.shape: (88766, 7)


In [5]:
print(df.shape)
print(df['contents'].isna().sum()) # 0

df.head()

(88766, 7)
0


,Unnamed: 0,source,search_words,url,title,contents,tagged_contents
0,0,블로그,아기+잠깨,https://blog.naver.com/srhymin/223091161466,NaN,저희 아기는 신생아때부터 청각과 촉각이 예민해 잠도 깊게 못 잘 뿐더러 잠투정이 정...,"[저희, 아기, 신생아, 때, 청각, 촉각, 잠도, 못, 뿐더러, 잠투, 정이, 정..."
1,1,블로그,아기+잠깨,https://blog.naver.com/sukm83/223091879514,NaN,정신없는 주방한켠 뒤죽박죽 알 수 없는 용기에 양념과 소스가 담겨져 있기에 주방이 ...,"[정신, 방한, 뒤죽박죽, 알, 수, 용기, 양념, 소스, 주방, 이번, 양념, 통..."
2,2,블로그,아기+잠깨,https://blog.naver.com/dldptmf5134/223096875782,NaN,안녕하세요 지유맘입니다! 오늘은 지유가 이유식과 유아식 그 중간즈음 토핑식을 할 때...,"[지유, 맘, 오늘, 지유, 이유식, 유아식, 그, 중간, 즈음, 토핑식, 때, 정..."
3,3,블로그,아기+잠깨,https://blog.naver.com/lia1100/223095799488,NaN,안녕하세요 라온엄마에요 오늘은 조금 특이한 아기 반찬을 준비했어요 방풍나물을 이용한...,"[엄마, 오늘, 조금, 아기, 반찬, 준비, 방, 나물, 이용, 방, 나물, 된장,..."
4,4,블로그,아기+잠깨,https://blog.naver.com/perfect_aj/223095824533,NaN,6 12개월 아기의 1일 권장 물섭취량은 220ml정도라고 한다. 음 우래기는.. ...,"[개월, 아기, 권장, 물섭취량, 정도, 음, 우리, 애기, 생각, 물, 물병, 번..."


In [6]:
df.drop(columns=['Unnamed: 0'], inplace=True)

## doc2vec 준비

In [7]:
# TaggedDocument 만들기
# doc2vec 태그 p_id

tagged_corpus_list = []
for n, i in tqdm(enumerate(df['tagged_contents'])):
    tag = f'document{n}'
    tagged_corpus_list.append(TaggedDocument(tags = [tag], words = i))

88766it [00:02, 44261.57it/s]


## doc2vec 학습시키기
    <model 하이퍼파라미터 값>
    # vector_size : 생성할 문서 벡터의 크기
    # alpha: 모델 학습시 초기 학습률 0.025는 일반적으로 사용됨
    # min_alpha: 학습 과정에서 alpha 값을 이 값으로 줄여나가게 됩니다
    # window : 문맥 윈도우 크기, 주변 몇개의 단어의 문맥을 고려하여 학습하는가

In [8]:
# 모델 선언
model_doc2vec = Doc2Vec(vector_size = 200, # 벡터 크기
                        alpha = 0.025, # 초기 학습률
                        min_alpha = 0.001, # 최소 학습률
                        window = 5, # 고려할 주변 단어 수
                        min_count = 5, # 최소 등장 횟수
                        epochs = 30,
                        dm = 1) # PV-DM

In [9]:
# 사전 생성
model_doc2vec.build_vocab(tagged_corpus_list)

# 학습하기
print(f"Training model for {model_doc2vec.epochs} epochs...")

total_loss = model_doc2vec.train(tagged_corpus_list,
                                 total_examples=model_doc2vec.corpus_count,
                                 epochs=model_doc2vec.epochs,
                                 compute_loss=True)

print("Training finished.")
print(f"Total training loss over {model_doc2vec.epochs} epochs: {total_loss}")

Training model for 30 epochs...
Training finished.
Total training loss over 30 epochs: None


## 단어 유사도 확인

- 코사인 유사도 점수 해석 기준
    - 코사인 유사도는 두 벡터 간의 각도의 코사인 값을 측정하며, -1에서 +1 사이의 값을 가집니다.
    - +1: 두 벡터가 정확히 같은 방향을 가리킴 (완벽하게 유사함).
    - 0: 두 벡터가 직교함 (관련성 없음, 독립적).
    - -1: 두 벡터가 정확히 반대 방향을 가리킴 (완벽하게 반대됨).
    - Word2Vec/Doc2Vec에서 most_similar()가 반환하는 코사인 유사도 점수를 "높다" 또는 "낮다"고 판단하는 절대적인 기준은 없습니다. <br>
    점수는 상대적이며 여러 요인에 따라 달라집니다.
    <br>
- 해석 시 고려해야 할 점:
    - 상대적 순위가 더 중요: most_similar() 결과에서 중요한 것은 절대적인 점수 값보다는 어떤 단어들이 상위에 랭크되는지 입니다. <br>
    의미론적으로 가장 관련성이 높은 단어들이 실제로 가장 높은 점수를 받고 상위에 위치하는지가 품질 판단의 핵심입니다.
    - 예시: '운동'의 유사 단어로 '헬스'(0.8), '스포츠'(0.75), '건강'(0.7) 등이 나왔다면 점수 차이는 있지만 모두 관련성이 높으므로 좋은 결과입니다. <br>
    만약 '운동'의 유사 단어로 '컴퓨터'(0.9), '요리'(0.85)가 나왔다면, 점수가 높아도 임베딩 품질이 나쁜 것입니다.
    <br>
- 맥락과 도메인:
    - 일반적인 단어 vs 전문 용어: 매우 자주 사용되는 일반적인 단어(예: '사람', '시간')는 다양한 문맥에서 사용되므로, <br>
    특정 동의어와의 유사도 점수가 상대적으로 낮게 나올 수 있습니다. <br>
    반면, 특정 도메인에서만 사용되는 전문 용어는 동의어와의 유사도 점수가 매우 높게 나올 수 있습니다.
    - 데이터 특성: 사용자 생성 비정형 텍스트는 노이즈가 많고 다양한 표현이 사용되므로, <br>
    잘 정제된 뉴스 기사 등으로 학습한 모델보다 전반적인 유사도 점수가 낮게 나올 수 있습니다.
    <br>
- 모델 파라미터:
    - vector_size: 차원이 낮으면 단어 간의 미묘한 차이를 구분하기 어려워 전반적으로 유사도가 높게 나올 수 있고, <br>
    차원이 너무 높으면 벡터 공간이 희소해져 유사도가 낮아질 수 있습니다.
    - window, epochs, min_count 등 다른 파라미터들도 학습된 벡터 공간의 구조에 영향을 미쳐 유사도 점수에 영향을 줍니다.
    <br>
- 단어 자체의 특성:
    - 동음이의어는 여러 의미의 문맥이 벡터에 혼합되어 특정 의미의 동의어와의 유사도가 낮아질 수 있습니다.
    - 매우 드물게 등장하는 단어(하지만 min_count는 넘은 단어)는 문맥 정보 부족으로 <br> 벡터 학습이 불안정하여 예상치 못한 유사도 결과를 보일 수 있습니다.

### 사전 확인

In [10]:
# 사전 확인: key_to_index 속성 사용 (추천 - 단어와 인덱스 매핑 확인)
# 모델 어휘 사전에 있는 단어들을 키로, 내부 인덱스를 값으로 갖는 딕셔너리

vocab_dict = model_doc2vec.wv.key_to_index
learned_vocab = list(vocab_dict.keys()) # 딕셔너리의 키(단어)들을 리스트로 변환

print(f"총 학습된 어휘 수: {len(learned_vocab)}")

print("\n학습된 어휘 샘플 (처음 20개):")
print(learned_vocab[:20])

print("\n학습된 어휘 샘플 (마지막 20개):")
print(learned_vocab[-20:])

총 학습된 어휘 수: 64941

학습된 어휘 샘플 (처음 20개):
['아기', '것', '때', '수', '낮잠', '엄마', '우리', '시간', '더', '아이', '집', '이', '나', '개월', '내', '육아', '날', '안', '거', '그']

학습된 어휘 샘플 (마지막 20개):
['송월타월', '클롭', '킬트', '무자격', '제소', '류트', '뇽발', '업쥐', '중화가', '고암면', '깨쥬', '세프', '부덕', '저냑', '인스피릿', '주호민', '쟁임템', '보증수표', '럽라', '르뱅쿠키']


In [12]:
# 어휘 사전과 빈도수 정보 가져오기
# get_vecattr 메서드를 사용하여 각 단어의 'count' 속성을 가져옵니다.
try:
    # 모든 단어 가져오기
    vocab_keys = list(model_doc2vec.wv.key_to_index.keys())

    # 각 단어와 해당 빈도수를 저장할 리스트 생성
    word_counts = []
    for word in vocab_keys:
        # get_vecattr로 'count' 속성 조회
        count = model_doc2vec.wv.get_vecattr(word, 'count')
        word_counts.append((word, count))

    # 빈도수(리스트 내 튜플의 두 번째 요소) 기준으로 내림차순 정렬
    sorted_word_counts = sorted(word_counts, key=lambda item: item[1], reverse=True)

    # 상위 20개 단어 추출
    top_n = 25
    top_words = sorted_word_counts[:top_n]

    print(f"--- 모델 어휘 사전 내 상위 {top_n}개 빈출 단어 (min_count={model_doc2vec.min_count} 이상) ---")
    for rank, (word, count) in enumerate(top_words, 1):
        print(f"{rank}. {word}: {count}")

except Exception as e:
    print(f"처리 중 오류 발생: {e}")

--- 모델 어휘 사전 내 상위 25개 빈출 단어 (min_count=5 이상) ---
1. 아기: 387908
2. 것: 294254
3. 때: 217223
4. 수: 205049
5. 낮잠: 203116
6. 엄마: 169687
7. 우리: 148172
8. 시간: 145856
9. 더: 143809
10. 아이: 138736
11. 집: 134757
12. 이: 134119
13. 나: 123246
14. 개월: 119988
15. 내: 114679
16. 육아: 114403
17. 날: 107878
18. 안: 107533
19. 거: 104001
20. 그: 102365
21. 오늘: 102342
22. 생각: 100434
23. 또: 97055
24. 저: 92280
25. 진짜: 90322


### 사전 검색

In [13]:
# 특정 단어가 사전에 있는지 확인
target_word = '아기'
if target_word in vocab_dict:
    print(f"\n단어 '{target_word}'는 어휘 사전에 포함되어 있습니다 (인덱스: {vocab_dict[target_word]}).")
else:
    print(f"\n단어 '{target_word}'는 어휘 사전에 없습니다 (min_count={model_doc2vec.min_count} 기준).")


단어 '아기'는 어휘 사전에 포함되어 있습니다 (인덱스: 0).


### 검색어와 유사어 목록 보기

In [34]:
# 특정 단어와 가장 유사한 단어 찾기
target_word = '수면' # 모델 어휘 사전에 있어야 함
top_n = 20

try:
    # most_similar 함수 호출
    # 입력: 유사도를 찾을 단어 (또는 단어 리스트)
    # topn: 반환할 유사 단어의 개수
    similar_words = model_doc2vec.wv.most_similar(target_word, topn=top_n)

    print(f"--- 단어 '{target_word}'와(과) 가장 유사한 단어 Top {top_n} ---")
    # 결과는 (유사단어, 코사인 유사도 점수) 튜플의 리스트 형태로 반환됨
    for word, similarity_score in similar_words:
        print(f"- {word}: {similarity_score:.4f}")

except KeyError:
    # target_word가 모델의 어휘 사전에 없는 경우 KeyError 발생
    print(f"오류: 단어 '{target_word}'가 모델의 어휘 사전에 없습니다.")
    print("다른 단어를 시도하거나, 학습 시 사용한 min_count 값을 확인해보세요.")
except Exception as e:
    print(f"오류 발생: {e}")

--- 단어 '수면'와(과) 가장 유사한 단어 Top 20 ---
- 숙면: 0.4784
- 통잠: 0.4604
- 혜민서: 0.4560
- 불안: 0.4420
- 식물인간: 0.4337
- 작졍: 0.4294
- 면시: 0.4259
- 전라북도교육청: 0.4248
- 밥상머리: 0.4131
- 관재: 0.4091
- 잠자리: 0.4056
- 노수: 0.4032
- 밤잠: 0.3998
- 맘스웰: 0.3906
- 놀잠: 0.3883
- 불안도: 0.3857
- 유청: 0.3822
- 투레: 0.3781
- 기지: 0.3709
- 수유: 0.3625


### 단어 조합과 유사어 찾기

In [36]:
# 여러 단어를 조합하여 유사한 단어 찾기 (긍정/부정 예시)
# 예: '왕' - '남자' + '여자' = '여왕' 유추와 유사한 방식
try:
    positive_words = ['아기', '수면', '낮잠', '밤잠'] # 긍정적으로 기여할 단어
    negative_words = ['참깨']        # 부정적으로 기여할 단어 (뺄셈 개념)

    # positive 리스트의 벡터는 더하고, negative 리스트의 벡터는 빼서 결과 벡터와 가장 유사한 단어 찾기
    derived_similar_words = model_doc2vec.wv.most_similar(positive=positive_words, negative=negative_words, topn=top_n)

    print(f"\n--- '{positive_words}' - '{negative_words}'와(과) 유사한 단어 Top {top_n} ---")
    for word, similarity_score in derived_similar_words:
        print(f"- {word}: {similarity_score:.4f}")

except KeyError as e:
    print(f"오류: 단어 '{e.args[0]}'가 모델의 어휘 사전에 없습니다.")
except Exception as e:
    print(f"오류 발생: {e}")


--- '['아기', '수면', '낮잠', '밤잠']' - '['참깨']'와(과) 유사한 단어 Top 20 ---
- 통잠: 0.5518
- 입면: 0.5365
- 잠: 0.5185
- 숙면: 0.4652
- 꿀잠: 0.4430
- 잠자리: 0.4325
- 수유: 0.4315
- 낮: 0.4261
- 취침: 0.4237
- 놀잠: 0.4143
- 목욕: 0.3999
- 쪽쪽이: 0.3965
- 스와들업: 0.3943
- 잠투정: 0.3931
- 아가: 0.3911
- 잠도: 0.3886
- 수면시간: 0.3866
- 이유식: 0.3750
- 밤수: 0.3705
- 푹: 0.3699


### 두 단어 유사도 계산

In [28]:
# 두 단어 간의 유사도 직접 계산하기
try:
    word1 = '낮잠'
    word2 = '밤잠'
    similarity = model_doc2vec.wv.similarity(word1, word2)
    print(f"\n--- '{word1}'와(과) '{word2}'의 코사인 유사도 ---")
    print(f"{similarity:.4f}")
except KeyError as e:
    print(f"오류: 단어 '{e.args[0]}'가 모델의 어휘 사전에 없습니다.")
except Exception as e:
    print(f"오류 발생: {e}")


--- '낮잠'와(과) '밤잠'의 코사인 유사도 ---
0.4340


### 벡터 값 데이터 프레임에 추가

* 벡터화 된 내용을 새로운 변수 vector 로 저장

In [29]:
vector_list = []
for i in tqdm(range(len(df))) :
    doc2vec = model_doc2vec.dv[f'document{i}']
    vector_list.append(doc2vec)

100%|██████████| 88766/88766 [00:00<00:00, 604574.67it/s]


In [30]:
df['vector'] = vector_list
df.head()

,source,search_words,url,title,contents,tagged_contents,vector
0,블로그,아기+잠깨,https://blog.naver.com/srhymin/223091161466,NaN,저희 아기는 신생아때부터 청각과 촉각이 예민해 잠도 깊게 못 잘 뿐더러 잠투정이 정...,"[저희, 아기, 신생아, 때, 청각, 촉각, 잠도, 못, 뿐더러, 잠투, 정이, 정...","[0.18305127, -1.2801502, -0.44998685, 0.799018..."
1,블로그,아기+잠깨,https://blog.naver.com/sukm83/223091879514,NaN,정신없는 주방한켠 뒤죽박죽 알 수 없는 용기에 양념과 소스가 담겨져 있기에 주방이 ...,"[정신, 방한, 뒤죽박죽, 알, 수, 용기, 양념, 소스, 주방, 이번, 양념, 통...","[3.5914354, -2.2061694, 1.5965343, 0.19793592,..."
2,블로그,아기+잠깨,https://blog.naver.com/dldptmf5134/223096875782,NaN,안녕하세요 지유맘입니다! 오늘은 지유가 이유식과 유아식 그 중간즈음 토핑식을 할 때...,"[지유, 맘, 오늘, 지유, 이유식, 유아식, 그, 중간, 즈음, 토핑식, 때, 정...","[0.3413881, -0.53281033, -1.3500248, 1.670243,..."
3,블로그,아기+잠깨,https://blog.naver.com/lia1100/223095799488,NaN,안녕하세요 라온엄마에요 오늘은 조금 특이한 아기 반찬을 준비했어요 방풍나물을 이용한...,"[엄마, 오늘, 조금, 아기, 반찬, 준비, 방, 나물, 이용, 방, 나물, 된장,...","[0.9519698, -1.2839632, -0.3893606, -0.1345413..."
4,블로그,아기+잠깨,https://blog.naver.com/perfect_aj/223095824533,NaN,6 12개월 아기의 1일 권장 물섭취량은 220ml정도라고 한다. 음 우래기는.. ...,"[개월, 아기, 권장, 물섭취량, 정도, 음, 우리, 애기, 생각, 물, 물병, 번...","[0.47277522, -0.9469169, 1.0451334, -0.1203481..."


In [31]:
# 저장하기

with open('vector_df_0510.pkl', 'wb') as f:
    pickle.dump(df, f)